In [1]:
import pandas as pd
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import psycopg2
import numpy as np
import matplotlib.pyplot as plt
def create_and_insert_product():
    connection = psycopg2.connect(
        host="localhost",
        port=5432,
        database="ZaislaiProducts",
        user="postgres",
        password="truputukas1982"
    )
    drop_table_query = '''
        DROP TABLE IF EXISTS butai233
'''
    create_table_query = """
        CREATE TABLE butai233 (
        id SERIAL PRIMARY KEY,
        Adresas VARCHAR(255),
        Kambariu_sk INT,
        Plotas DECIMAL(10,2),
        Aukstas VARCHAR(255),
        Kaina DECIMAL(10,2),
        Kv_kaina DECIMAL(10,2)
        )
    """
    cursor = connection.cursor()
    cursor.execute(drop_table_query)
    cursor.execute(create_table_query)
    print('Table created successfully')
    Butu_sarasas = []
    for i in range(1,6):
        url = f'https://www.aruodas.lt/atviru-duru-dienos/puslapis/{i}/'
        response = requests.get(url)
        # print(response.content)
        soup = BeautifulSoup(response.content, 'html.parser')
        blokas = soup.find_all('div', class_='list-row-v2 object-row opendoor advert')
        for butas in blokas:
            Adresas = soup.select_one('div.list-adress-v2 h3').text.strip()
            Kambariu_sk = butas.find('div', class_='list-RoomNum-v2 list-detail-v2').text.strip()
            Plotas = butas.find('div', class_='list-AreaOverall-v2 list-detail-v2').text.strip()
            Aukstas = butas.find('div', class_='list-Floors-v2 list-detail-v2').text.strip()
            Kaina = butas.find('span', class_='list-item-price-v2').text.strip('€').replace(' ', '')
            Kv_kaina = butas.find('span', class_='price-pm-v2').text.strip().replace('€/m²', '').replace(' ', '').replace(',', '.')
            if Kambariu_sk == '' or Plotas == '' or Kaina == '' or Kv_kaina == '':
                continue
            Kambariu_sk = int(Kambariu_sk)
            Plotas = float(Plotas)
            Kaina = int(Kaina)
            Kv_kaina = float(Kv_kaina)
            Butu_sarasas.append((Adresas, Kambariu_sk, Plotas, Aukstas, Kaina, Kv_kaina))
            # print(Butu_sarasas)
    df = pd.DataFrame(Butu_sarasas, columns=['Adresas', 'Kambarius_sk', 'Plotas', 'Aukstas', 'Kaina', 'Kv_kaina'])
        # print(df)
    df.to_csv('Butai22.csv')
    insert_query = "INSERT INTO butai233 (Adresas, Kambariu_sk, Plotas, Aukstas, Kaina, Kv_kaina) VALUES(%s, %s,%s,%s,%s,%s)"
    cursor.executemany(insert_query, Butu_sarasas)
    print(f'Products inserted into list succesfully!')
    connection.commit()
        # cursor.close()
        # connection.close()
    select_query = "SELECT * FROM butai233"
        # cursor.execute(select_query)
        # Butu_sarasas = cursor.fetchall()
    df = pd.DataFrame(Butu_sarasas, columns = ['Adresas', 'Kambarius_sk', 'Plotas', 'Aukstas', 'Kaina', 'Kv_kaina'])
    print(df)
if __name__ == '__main__':
    create_and_insert_product()

Table created successfully
Products inserted into list succesfully!
                            Adresas  Kambarius_sk  Plotas Aukstas   Kaina  \
0               PalangaVanagupės g.             1   34.62     4/4  117500   
1               PalangaVanagupės g.             3   71.24     2/4  241000   
2               PalangaVanagupės g.             1   33.85     1/4  116800   
3               PalangaVanagupės g.             2   44.28     2/5  138000   
4               PalangaVanagupės g.             2   44.00     2/4  161506   
..                              ...           ...     ...     ...     ...   
77  Vilnius, NaujamiestisSkroblų g.             3   55.53     3/3  129999   
78  Vilnius, NaujamiestisSkroblų g.             3   63.22     4/4  119990   
79  Vilnius, NaujamiestisSkroblų g.             2   38.00     1/2  145000   
80  Vilnius, NaujamiestisSkroblų g.             2   58.54     3/3  307000   
81  Vilnius, NaujamiestisSkroblų g.             2   51.66     5/7  146900   

    Kv_